In [10]:
import numpy as np
#import numba as nb
import chardet
import math
import csv

In [11]:
def detect_encoding(file_path):
    """
    Detect the encoding of the given CSV file.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    str: The detected encoding of the file.
    """
    with open(file_path, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))
        return result['encoding']

def read_csv_with_encoding(file_path, columns_to_keep=None):
    """
    Read the CSV file with the detected encoding and keep specified columns.

    Parameters:
    file_path (str): The path to the CSV file.
    columns_to_keep (list, optional): A list of indices of columns to keep. 
                                      If None, all columns will be kept.

    Returns:
    list: A list of rows with only the specified columns (or all columns if none specified).
    """
    encoding = detect_encoding(file_path)
    print(f"\nDetected encoding: {encoding}")

    data = []
    with open(file_path, 'r', encoding=encoding) as file:
        reader = csv.reader(file)
        header = next(reader)  # Read the header to determine the number of columns
        max_columns = len(header)

        # If no columns specified, keep all columns
        if columns_to_keep is None:
            columns_to_keep = list(range(max_columns))

        for row in reader:
            selected_row = [row[i] for i in columns_to_keep]
            data.append(selected_row)
    
    return data

# Function to find the index of a specific value in a given column
def find_index(column_data, value):
    for index, item in enumerate(column_data):
        if item == value:
            return index
    return None

def convert_to_type(data, dtype=float):
    if dtype not in [float, int]:
        raise ValueError("dtype must be either 'float' or 'int'")
    
    converted_data = []
    for value in data:
        try:
            converted_data.append(dtype(value))
        except ValueError:
            print(f"Warning: Could not convert '{value}' to {dtype}")
            converted_data.append(None)  # Optionally, handle conversion errors
    
    return converted_data

def printAny(data, column_name="Unnamed"):
    print("")
    length = len(data)
    
    # Determine the data type
    data_type = type(data[0]).__name__ if length > 0 else "Unknown"

    if length > 10:
        for i in range(5):
            print(f"{i}       {data[i]}")
        print("       ...")
        for i in range(length-5, length):
            print(f"{i}    {data[i]}")
    else:
        for index, value in enumerate(data):
            print(f"{index}       {value}")
    
    print(f"Name: {column_name}, Length: {length}, dtype: {data_type}")


def printVec(data, column_name="Unnamed"):
    print("")  # Optional: print a newline for formatting
    length = len(data)
    
    # Determine the data type
    data_type = type(data[0]).__name__ if length > 0 else "Unknown"

    if length > 10:
        for i in range(5):
            value = data[i]
            try:
                print(f"{i}       {float(value):.6f}")
            except ValueError:
                print(f"{i}       {value}")
        print("       ...")
        for i in range(length-5, length):
            value = data[i]
            try:
                print(f"{i}    {float(value):.6f}")
            except ValueError:
                print(f"{i}    {value}")
    else:
        for index, value in enumerate(data):
            try:
                print(f"{index}       {float(value):.6f}")
            except ValueError:
                print(f"{index}       {value}")

    print(f"Name: {column_name}, Length: {length}, dtype: {data_type}")

def cumsum(values):
    cum_sum = []
    total = 0
    for value in values:
        total += value
        cum_sum.append(total)
    return cum_sum

def calculate_exp(x, terms=20):
    result = 1.0
    term = 1.0
    for n in range(1, terms):
        term *= x / n
        result += term
    return 

def Psat_WV(T_K):
    """
    Water vapour saturation pressure
    W. Wagner and A. Pruß:" The IAPWS Formulation 1995 for the
    Thermodynamic Properties of Ordinary Water Substance for General and Scientific Use ",
    Journal of Physical and Chemical Reference Data, June 2002 ,Volume 31, Issue 2, pp.
    387535)

    Returns Saturation vapor pressure (hPa)
    """
    Tc = 647.096  # Critical temperature, K
    Pc = 220640   # Critical pressure, hPa
    
    C1 = -7.85951783
    C2 = 1.84408259
    C3 = -11.7866497
    C4 = 22.6807411
    C5 = -15.9618719
    C6 = 1.80122502
    
    teta = 1 - T_K / Tc
    
    x = Tc / T_K * (C1 * teta + C2 * teta ** 1.5 + C3 * teta ** 3 + C4 * teta ** 3.5 + C5 * teta ** 4 + C6 * teta ** 7.5)
    
    x = math.exp(x) * Pc
    
    return x

In [12]:
# Detect the encoding of the file
with open('Snow_Storage_Data.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
    encoding = result['encoding']

print(encoding)

# Define the indices of the columns you want to keep
columns_to_keep = list(range(18))

# Read the CSV file with the detected encoding and load the data into a NumPy array
data = np.genfromtxt('Snow_Storage_Data.csv', delimiter=',', dtype=str, encoding=encoding, skip_header=1, usecols=columns_to_keep)

# Display the first column of each row
print(data[:5, 9])  # Column 9 - YEAR column

# Find the index where the first NaN value appears in the 'YEAR.1' column (assuming it's the 9th column)
first_nan_index = np.where(data[:, 9] == '')[0][0] if np.any(data[:, 9] == '') else None

print(first_nan_index)

# Get '2023-04-01T00:00' format time data from the 7th column
time_column = data[:, 6]
print(time_column[:5])  # Display the first 5 entries for verification


ISO-8859-1
['MO' '1' '1' '1' '1']
8737
['time' '2023-01-01T00:00' '2023-01-01T01:00' '2023-01-01T02:00'
 '2023-01-01T03:00']


In [13]:
# Convert find_index function to use NumPy
def npfind_index(array, value):
    return np.where(array == value)[0][0]

In [14]:
# Find the start and end indices
period_start_in = npfind_index(time_column, '2023-04-01T00:00')
period_end_in = npfind_index(time_column, '2023-08-31T23:00')

# Slice the data to include only the desired period
rdata = data[period_start_in:period_end_in + 1]  # rdata - "ranged data"

# Display the time column for verification
print(time_column)

# Display the sliced data for verification
print(rdata)

# Get air temperature data and assign it to a vector
air_temp_vec = rdata[:, 7].astype(float)
printVec(air_temp_vec, column_name="Air temperature (Celsius)")

# Get air velocity data and assign it to a vector
air_vel_vec = rdata[:, 15].astype(float)
printVec(air_vel_vec, column_name="Air velocity (m/s)")

# Extract the amount of precipitation column from the data
prec_vec = rdata[:, 5].astype(float)
printVec(prec_vec, column_name="Precipitation (m/h)")

# Extract the global solar irradiance (W/m2) column from the data
glob_solir_vec = rdata[:, 12].astype(float)
printVec(glob_solir_vec, column_name="Global solar irradiance (W/m2)")


['time' '2023-01-01T00:00' '2023-01-01T01:00' ... '2023-12-31T21:00'
 '2023-12-31T22:00' '2023-12-31T23:00']
[['2023' '4' '1' ... '6.0' '2023-04-01T00:00' '72']
 ['2023' '4' '1' ... '6.1' '2023-04-01T01:00' '75']
 ['2023' '4' '1' ... '6.5' '2023-04-01T02:00' '65']
 ...
 ['2023' '8' '31' ... '3.8' '2023-08-31T21:00' '91']
 ['2023' '8' '31' ... '3.6' '2023-08-31T22:00' '91']
 ['2023' '8' '31' ... '2.8' '2023-08-31T23:00' '93']]

0       0.200000
1       0.100000
2       -0.100000
3       -0.200000
4       -0.400000
       ...
3667    17.500000
3668    17.100000
3669    16.900000
3670    16.500000
3671    15.800000
Name: Air temperature (Celsius), Length: 3672, dtype: float64

0       6.000000
1       6.100000
2       6.500000
3       6.500000
4       7.000000
       ...
3667    4.000000
3668    4.100000
3669    3.800000
3670    3.600000
3671    2.800000
Name: Air velocity (m/s), Length: 3672, dtype: float64

0       0.000000
1       0.000010
2       0.000010
3       0.000010
4       0.00

In [15]:
# Detect the encoding of the file
with open('Snow_Storage_Data.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
    encoding = result['encoding']

print(encoding)

# Create an empty list to store the data
data = []

# Define the indices of the columns you want to keep
columns_to_keep = []
for i in range(18):
    columns_to_keep.append(i)  

# Read the CSV file with the detected encoding
with open('Snow_Storage_Data.csv', 'r', encoding=encoding) as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip the first row (header)
    for row in reader:
        selected_row = [row[i] for i in columns_to_keep]
        data.append(selected_row)

# Display the first column of each row
for row in data[:5]:
    print(row[9])   # row[9] - YEAR column

# Find the index where the first NaN value appears in the 'YEAR.1' column (assuming it's the 5th column, change if needed)
first_nan_index = None
for index, row in enumerate(data):
    try:
        if row[9].strip() == '':
            first_nan_index = index
            break
    except IndexError:
        continue

print(first_nan_index)

# Get '2023-04-01T00:00' format time data from 7th column
time_column = [row[6] for row in data]

# Find the start and end indices
period_start_in = find_index(time_column, '2023-04-01T00:00')
period_end_in   = find_index(time_column, '2023-08-31T23:00')

# Slice the data to include only the desired period
#df_wo_nan_period = data[period_start_in:period_end_in + 1]  # +1 to include the end index

# Display the sliced data
#for row in data[period_start_in:period_end_in]:
#    print(row[7])

printVec(time_column)
rdata = data[period_start_in:period_end_in+1] # rdata - "ranged data"
printAny(rdata, column_name="All data")

# Get air temperature data and assign it to a vector
air_temp_vec_raw = [row[7] for row in rdata]
air_temp_vec = convert_to_type(air_temp_vec_raw, dtype=float)
printVec(air_temp_vec, column_name="Air temperature (Celsius)")

# Get air velocity data and assign it to a vector
air_vel_vec_raw = [row[15] for row in rdata]
air_vel_vec = convert_to_type(air_vel_vec_raw, dtype=float)
printVec(air_vel_vec, column_name="Air velocity (m/s)")

# Extract the amount of precipitation column from the data
prec_vec_raw = [row[5] for row in rdata]
prec_vec = convert_to_type(prec_vec_raw, dtype=float)
printVec(prec_vec, column_name="Precipitation (m/h)")

# Extract the global solar irradiance (W/m2) column from the data
glob_solir_vec_raw = [row[12] for row in rdata]
glob_solir_vec = convert_to_type(glob_solir_vec_raw, dtype=float)
printVec(glob_solir_vec, column_name="Global solar irradiance (W/m2)")

ISO-8859-1
MO
1
1
1
1
8737

0       time
1       2023-01-01T00:00
2       2023-01-01T01:00
3       2023-01-01T02:00
4       2023-01-01T03:00
       ...
8756    2023-12-31T19:00
8757    2023-12-31T20:00
8758    2023-12-31T21:00
8759    2023-12-31T22:00
8760    2023-12-31T23:00
Name: Unnamed, Length: 8761, dtype: str

0       ['2023', '4', '1', '0', '0', '0', '2023-04-01T00:00', '0.2', '2023', '4', '1', '0', '0', '2023-04-01T00:00', '21.6', '6.0', '2023-04-01T00:00', '72']
1       ['2023', '4', '1', '1', '0.01', '0.00001', '2023-04-01T01:00', '0.1', '2023', '4', '1', '1', '0', '2023-04-01T01:00', '22', '6.1', '2023-04-01T01:00', '75']
2       ['2023', '4', '1', '2', '0.01', '0.00001', '2023-04-01T02:00', '-0.1', '2023', '4', '1', '2', '0', '2023-04-01T02:00', '23.4', '6.5', '2023-04-01T02:00', '65']
3       ['2023', '4', '1', '3', '0.01', '0.00001', '2023-04-01T03:00', '-0.2', '2023', '4', '1', '3', '0', '2023-04-01T03:00', '23.4', '6.5', '2023-04-01T03:00', '77']
4       ['2023', '4', '

In [16]:
# Heat transfer coefficient at the external surface
h = 22.7 # W/(m^2K)
# Solar light absorptivity
alpha = 0.8
# Correction factor for horizontal surface
T_cor_fact = 4.0 # °C

In [17]:
#@nb.njit
#def compute_sol_air(alpha, glob_solir_vec, h, air_temp_vec, T_cor_fact):
#    T_sol_air_vec = np.empty(len(air_temp_vec))
#    for i in range(len(air_temp_vec)):
#        T_sol_air_vec[i] = alpha * glob_solir_vec[i] / h + air_temp_vec[i] - T_cor_fact
#    return T_sol_air_vec

In [18]:
#%%timeit
## Example usage
#T_sol_air_vec = compute_sol_air(alpha, glob_solir_vec, h, air_temp_vec, T_cor_fact)

In [19]:
%%timeit
T_sol_air_vec = []
for i in range(len(air_temp_vec)):
    T_sol_air_vec.append(alpha * glob_solir_vec[i] / h + air_temp_vec[i] - T_cor_fact)

294 μs ± 6.81 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
# Convert lists to NumPy arrays if they aren't already
glob_solir_vec = np.array(glob_solir_vec)
air_temp_vec = np.array(air_temp_vec)

In [21]:
%%timeit
# Calculate T_sol_air_vec2 using NumPy
T_sol_air_vec2 = alpha * glob_solir_vec / h + air_temp_vec - T_cor_fact

# Print the result for verification
#print(T_sol_air_vec2)

6.21 μs ± 68.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
# Alternative version:
# Improved efficiency with list comprehension
T_sol_air_vec2 = [
    alpha * glob_solir / h + air_temp - T_cor_fact 
    for glob_solir, air_temp in 
    zip(glob_solir_vec, air_temp_vec)
]
#printVec(T_sol_air_vec2, column_name="Solar-Air (Celsius)")

1.04 ms ± 20.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
